In [1]:
import numpy as np
import os
import pandas as pd
import copy 
import datetime 
data_path = '../../../code_data/shanghai_t2dm/'
process_file = 'nips_23'

# pid_list = [1000 + d for d in range(1, 13)]


In [2]:
from xls2xlsx import XLS2XLSX
import shutil


for x in os.listdir(os.path.join(data_path, 'raw_data')):
    print(x)
    if not os.path.exists(os.path.join(data_path, 'processing/temp1')):
        os.makedirs(os.path.join(data_path, 'processing/temp1'))
    if x.endswith(".xls"):
        XLS2XLSX(os.path.join(data_path, 'raw_data', x)).to_xlsx(os.path.join(data_path, 'processing/temp1', x+'x'))
    elif x.endswith('.xlsx'):
        shutil.copyfile(os.path.join(data_path, 'raw_data', x), os.path.join(data_path, 'processing/temp1', x))

2000_0_20201230.xlsx
2001_0_20201102.xlsx
2001_1_20201117.xlsx
2002_0_20210513.xlsx
2003_0_20210615.xlsx
2004_0_20211028.xlsx
2005_0_20211201.xlsx
2006_0_20211112.xlsx
2007_0_20220108.xlsx
2008_0_20220118.xlsx
2009_0_20211103.xlsx
2010_0_20220111.xlsx
2011_0_20220123.xlsx
2012_0_20220126.xlsx
2013_0_20220123.xlsx
2014_0_20201224.xlsx
2014_1_20210317.xlsx
2015_0_20210203.xlsx
2015_1_20210219.xlsx
2016_0_20201224.xls
2017_0_20210102.xlsx
2017_1_20201118.xls
2018_0_20210420.xls
2019_0_20210513.xls
2020_0_20210423.xls
2021_0_20211013.xls
2022_0_20210419.xls
2023_0_20210812.xls
2024_0_20210429.xls
2025_0_20210506.xls
2026_0_20210916.xls
2027_0_20210521.xls
2028_0_20210426.xls
2029_0_20210526.xls
2030_0_20210531.xls
2031_0_20210601.xls
2032_0_20210727.xls
2033_0_20210615.xls
2034_0_20210624.xls
2035_0_20210629.xls
2036_0_20210803.xls
2037_0_20201221.xls
2038_0_20210608.xls
2039_0_20210615.xls
2040_0_20210729.xls
2041_0_20210813.xls
2042_0_20211019.xls
2043_0_20210513.xls
2044_0_20211101.xls


In [3]:
names = ['ts', 'glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',	'insulin_iv',] 
usecols = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
pid2df = {}
for x in os.listdir(os.path.join(data_path,
                                  'processing/temp1')):
    if x.endswith('.xlsx'):
        # Prints only text file present in My Folder
        print(x)
        pid = int(x[:4])
        df = pd.read_excel(os.path.join(data_path, 'processing/temp1',x), names=names, usecols=usecols)
        if pid not in pid2df:
            pid2df[pid] = df
        else:
            pid2df[pid] = pd.concat([pid2df[pid], df])

2000_0_20201230.xlsx
2001_0_20201102.xlsx
2001_1_20201117.xlsx
2002_0_20210513.xlsx
2003_0_20210615.xlsx
2004_0_20211028.xlsx
2005_0_20211201.xlsx
2006_0_20211112.xlsx
2007_0_20220108.xlsx
2008_0_20220118.xlsx
2009_0_20211103.xlsx
2010_0_20220111.xlsx
2011_0_20220123.xlsx
2012_0_20220126.xlsx
2013_0_20220123.xlsx
2014_0_20201224.xlsx
2014_1_20210317.xlsx
2015_0_20210203.xlsx
2015_1_20210219.xlsx
2016_0_20201224.xlsx
2017_0_20210102.xlsx
2017_1_20201118.xlsx
2018_0_20210420.xlsx
2019_0_20210513.xlsx
2020_0_20210423.xlsx
2021_0_20211013.xlsx
2022_0_20210419.xlsx
2023_0_20210812.xlsx
2024_0_20210429.xlsx
2025_0_20210506.xlsx
2026_0_20210916.xlsx
2027_0_20210521.xlsx
2028_0_20210426.xlsx
2029_0_20210526.xlsx
2030_0_20210531.xlsx
2031_0_20210601.xlsx
2032_0_20210727.xlsx
2033_0_20210615.xlsx
2034_0_20210624.xlsx
2035_0_20210629.xlsx
2036_0_20210803.xlsx
2037_0_20201221.xlsx
2038_0_20210608.xlsx
2039_0_20210615.xlsx
2040_0_20210729.xlsx
2041_0_20210813.xlsx
2042_0_20211019.xlsx
2043_0_202105

In [4]:
import re
def change_meal(x):
    if pd.notna(x):
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        return np.array(result, dtype=float).sum()
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].apply(change_meal)

In [5]:
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].astype(np.float64)

In [6]:
def change_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+\s*IU')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan

        result = [res[:-3] for res in result]
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].apply(change_insulin)
    # pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_insulin)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].apply(change_insulin)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].apply(change_insulin)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].apply(change_insulin)

In [7]:
def nan_sum(x):
        if pd.notna(x[0]) and pd.notna(x[1]):
            return x[0] + x[1]
        elif pd.notna(x[0]):
            return x[0]
        elif pd.notna(x[1]):
            return x[1]
        else:
            return np.NaN
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].astype(np.float64)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].astype(np.float64)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].astype(np.float64)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].astype(np.float64)
    pid2df[pid]['bolus'] = pid2df[pid][['insulin_dose_sc', 'insulin_bolus']].apply(nan_sum, axis=1)
    

In [8]:
def change_non_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_non_insulin)


In [9]:
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].astype(np.float64)

In [10]:
pid2regular_df = {}
pid_list = list(pid2df.keys())
for pid in pid_list:
    df = pid2df[pid]
    df = df.set_index('ts')
    df = df.sort_index()
    pid2regular_df[pid] = df
    print(f'{pid}, total_points:{len(df)}, start:{df.index[0]}, end:{df.index[-1]}')


2000, total_points:1339, start:2020-12-30 13:54:00, end:2021-01-13 12:24:00
2001, total_points:2534, start:2020-11-02 09:40:00, end:2020-12-01 07:49:00
2002, total_points:571, start:2021-05-13 16:22:00, end:2021-05-19 14:52:00
2003, total_points:585, start:2021-06-15 17:20:00, end:2021-06-21 19:20:00
2004, total_points:484, start:2021-10-28 14:28:00, end:2021-11-02 15:13:00
2005, total_points:691, start:2021-12-01 15:55:00, end:2021-12-08 20:25:00
2006, total_points:480, start:2021-11-12 15:20:00, end:2021-11-17 15:05:00
2007, total_points:831, start:2022-01-08 16:46:00, end:2022-01-17 08:16:00
2008, total_points:533, start:2022-01-18 16:29:00, end:2022-01-24 05:29:00
2009, total_points:494, start:2021-11-03 09:43:00, end:2021-11-08 12:58:00
2010, total_points:539, start:2022-01-11 17:44:00, end:2022-01-17 08:14:00
2011, total_points:462, start:2022-01-23 16:58:00, end:2022-01-28 12:13:00
2012, total_points:288, start:2022-01-26 12:33:00, end:2022-01-29 12:18:00
2013, total_points:350,

In [11]:
# check
for pid in pid_list:
    delta_ts = pid2regular_df[pid].index[1:] - pid2regular_df[pid].index[:-1]
    print(pid, (delta_ts>datetime.timedelta(minutes=15, seconds=10)).sum())

2000 0
2001 1
2002 0
2003 0
2004 0
2005 0
2006 0
2007 0
2008 0
2009 0
2010 0
2011 0
2012 0
2013 0
2014 1
2015 1
2016 0
2017 1
2018 0
2019 0
2020 0
2021 0
2022 0
2023 1
2024 0
2025 0
2026 0
2027 0
2028 0
2029 0
2030 0
2031 0
2032 0
2033 0
2034 0
2035 0
2036 0
2037 0
2038 0
2039 0
2040 1
2041 0
2042 0
2043 0
2044 0
2045 0
2046 0
2047 0
2048 0
2049 0
2050 0
2051 0
2052 0
2053 0
2054 0
2055 1
2056 0
2057 0
2058 0
2059 0
2060 0
2061 0
2062 0
2063 0
2064 0
2065 0
2066 0
2067 0
2068 0
2069 2
2070 0
2071 0
2072 0
2073 0
2074 1
2075 0
2076 0
2077 0
2078 1
2079 0
2080 2
2081 0
2082 0
2083 0
2084 1
2085 0
2086 0
2087 0
2088 0
2089 0
2090 0
2091 0
2092 0
2093 0
2094 0
2095 5
2096 0
2097 0
2098 0
2099 0


In [12]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().loc[:, 'ts'].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        
    return pid2data

In [13]:
pid2regular_df = add_time_attributes(pid2regular_df)


In [14]:
# add discrete y

def discrete(y):
    if y < 70:
        return 0
    elif 70<= y <=180:
        return 1
    elif y > 180:
        return 2
    else:
        return np.nan


def add_discrete_y(pid2data):
    for pid in pid2data:
        data = pid2data[pid]
        data['discrete_y'] = data['glucose_level']

        data['discrete_y'] = data['discrete_y'].apply(discrete)
        
    return pid2data

In [15]:
pid2regular_df = add_discrete_y(pid2regular_df)

In [16]:
def add_indicate_for_nan(pid2data, ignore_indicate_set):
    for pid in pid2data:
        data = pid2data[pid]
        for attri in data:
            if attri in ignore_indicate_set:
                continue
            data[attri + '_indicate'] = 1
            data.loc[pd.notna(data[attri]), attri + '_indicate'] = 0
            if sum(data[attri + '_indicate']) != len(data[attri]) - sum(pd.notna(data[attri])):
                print('error')
    return pid2data

In [17]:
ignore_indicate_set = ['day_of_week', 'hour', 'minute',
       'timestamp', 'sin_time', 'cos_time',]
pid2regular_df = add_indicate_for_nan(pid2regular_df, ignore_indicate_set)

In [18]:
pid2train = {}
pid2valid = {}
pid2test = {}

for pid in pid_list:
    idxs = pid2regular_df[pid].index[pd.notna(pid2regular_df[pid]['glucose_level'])]

    idxs = idxs.to_frame()

    total_len = len(idxs)
    train_end = int(total_len * 0.6)
    valid_end = int(total_len * 0.8) 

    pid2train[pid] = pid2regular_df[pid].loc[:idxs.iloc[train_end][0]]

    pid2valid[pid] = pid2regular_df[pid].loc[idxs.iloc[train_end + 1][0]:idxs.iloc[valid_end][0]]

    pid2test[pid] = pid2regular_df[pid].loc[idxs.iloc[valid_end + 1][0]:]

    print(pid, total_len, len(pid2train[pid]), len(pid2valid[pid]), len(pid2test[pid]))

2000 1339 804 268 267
2001 2534 1521 507 506
2002 571 343 114 114
2003 585 352 117 116
2004 484 291 97 96
2005 691 415 138 138
2006 480 289 96 95
2007 831 499 166 166
2008 533 320 107 106
2009 494 297 99 98
2010 539 324 108 107
2011 462 278 92 92
2012 288 173 58 57
2013 350 211 70 69
2014 2672 1604 534 534
2015 2657 1595 531 531
2016 1339 804 268 267
2017 2674 1605 535 534
2018 1231 739 246 246
2019 751 451 150 150
2020 539 324 108 107
2021 1135 682 227 226
2022 945 568 189 188
2023 541 325 108 108
2024 729 438 146 145
2025 761 457 152 152
2026 1136 682 227 227
2027 1133 680 227 226
2028 823 494 165 164
2029 741 445 148 240
2030 946 568 189 189
2031 922 554 184 184
2032 281 169 56 56
2033 726 436 145 145
2034 387 233 77 77
2035 247 149 49 49
2036 531 319 106 106
2037 1338 803 268 267
2038 546 328 109 109
2039 633 380 127 126
2040 1024 615 205 204
2041 392 236 78 78
2042 400 241 80 79
2043 825 496 165 164
2044 641 385 128 128
2045 1339 804 268 267
2046 1147 689 229 229
2047 1217 731 243

In [19]:
for pid in pid2test:
    print(pid, 'blood_ketone', pd.notna(pid2test[pid]['blood_ketone']).sum())
    print(pid,'insulin_iv', pd.notna(pid2test[pid]['insulin_iv']).sum())

2000 blood_ketone 0
2000 insulin_iv 0
2001 blood_ketone 0
2001 insulin_iv 0
2002 blood_ketone 0
2002 insulin_iv 0
2003 blood_ketone 0
2003 insulin_iv 0
2004 blood_ketone 0
2004 insulin_iv 0
2005 blood_ketone 0
2005 insulin_iv 0
2006 blood_ketone 0
2006 insulin_iv 0
2007 blood_ketone 0
2007 insulin_iv 0
2008 blood_ketone 0
2008 insulin_iv 0
2009 blood_ketone 0
2009 insulin_iv 0
2010 blood_ketone 0
2010 insulin_iv 0
2011 blood_ketone 0
2011 insulin_iv 0
2012 blood_ketone 0
2012 insulin_iv 0
2013 blood_ketone 0
2013 insulin_iv 0
2014 blood_ketone 0
2014 insulin_iv 0
2015 blood_ketone 0
2015 insulin_iv 0
2016 blood_ketone 0
2016 insulin_iv 0
2017 blood_ketone 0
2017 insulin_iv 0
2018 blood_ketone 0
2018 insulin_iv 0
2019 blood_ketone 0
2019 insulin_iv 0
2020 blood_ketone 0
2020 insulin_iv 0
2021 blood_ketone 0
2021 insulin_iv 0
2022 blood_ketone 0
2022 insulin_iv 0
2023 blood_ketone 0
2023 insulin_iv 0
2024 blood_ketone 0
2024 insulin_iv 0
2025 blood_ketone 0
2025 insulin_iv 0
2026 blood_k

In [20]:
pid2train[pid].columns

Index(['glucose_level', 'finger_stick', 'blood_ketone', 'meal',
       'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',
       'insulin_iv', 'bolus', 'day_of_week', 'hour', 'minute', 'timestamp',
       'sin_time', 'cos_time', 'discrete_y', 'glucose_level_indicate',
       'finger_stick_indicate', 'blood_ketone_indicate', 'meal_indicate',
       'insulin_dose_sc_indicate', 'hypo_agents_indicate',
       'insulin_bolus_indicate', 'basal_indicate', 'insulin_iv_indicate',
       'bolus_indicate', 'discrete_y_indicate'],
      dtype='object')

In [21]:
# calculate pearson
df = pd.concat(list(pid2train.values()))
df[['glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal', 'insulin_iv','timestamp']].corr(method ='pearson')['glucose_level'].sort_values()


blood_ketone      -0.554998
insulin_bolus     -0.188281
insulin_dose_sc   -0.124295
hypo_agents       -0.091864
meal               0.006124
basal              0.133803
timestamp          0.171006
insulin_iv         0.676332
finger_stick       0.777724
glucose_level      1.000000
Name: glucose_level, dtype: float64

In [22]:
pid_attri2mean_std = {}
attris_should_norm = ['glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',	'insulin_iv', 'bolus', 'timestamp']
for pid in pid_list:
    for attri in attris_should_norm:
        mean = pid2train[pid][attri].mean()
        std = pid2train[pid][attri].std()
        if std is np.NaN or std is pd.NaT or std == 0:
            std = 1e-6
        pid_attri2mean_std[(pid, attri)] = (mean, std)

if not os.path.exists(os.path.join(data_path, process_file)):
    os.makedirs(os.path.join(data_path, process_file))
np.save(os.path.join(data_path, process_file, 'pid_attri2mean_std.npy'), pid_attri2mean_std)

In [23]:
def norm_data(pid2data, pid_attri2mean_std, attris_should_norm):

    new_pid2data = {}
    for pid in pid2data:

        new_pid2data[pid] = copy.deepcopy(pid2data[pid]) 

        for attri in attris_should_norm:

            mean, std = pid_attri2mean_std[(pid, attri)]

            if std <= 1e-6:
                new_pid2data[pid][attri] = new_pid2data[pid][attri] / mean
            else:
                new_pid2data[pid][attri] = (new_pid2data[pid][attri] - mean) / std          

    return new_pid2data

In [24]:
pid2train_norm = norm_data(pid2train, pid_attri2mean_std, attris_should_norm)
pid2valid_norm = norm_data(pid2valid, pid_attri2mean_std, attris_should_norm)
pid2test_norm = norm_data(pid2test, pid_attri2mean_std, attris_should_norm)

In [25]:
attri_list_wo_glucose = [
    'finger_stick', 'blood_ketone', 'meal',
    'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',
    'insulin_iv', 'bolus', 'day_of_week', 'hour', 'minute', 'timestamp',
    'sin_time', 'cos_time', 'discrete_y', 'glucose_level_indicate',
    'finger_stick_indicate', 'blood_ketone_indicate', 'meal_indicate',
    'insulin_dose_sc_indicate', 'hypo_agents_indicate',
    'insulin_bolus_indicate', 'basal_indicate', 'insulin_iv_indicate', 'bolus_indicate'
]

temp = []

for idx, attri in enumerate(attri_list_wo_glucose):
    temp.append({'attri':attri, 'idx':idx}, )
attri2idx = pd.DataFrame(temp).set_index('attri')
idx2attri = pd.DataFrame(temp).set_index('idx')
print(attri2idx)
print(idx2attri)

save_path = os.path.join(data_path, process_file)

attri2idx.to_pickle(os.path.join(save_path,'attri2idx.pkl'))
idx2attri.to_pickle(os.path.join(save_path,'idx2attri.pkl'))


                          idx
attri                        
finger_stick                0
blood_ketone                1
meal                        2
insulin_dose_sc             3
hypo_agents                 4
insulin_bolus               5
basal                       6
insulin_iv                  7
bolus                       8
day_of_week                 9
hour                       10
minute                     11
timestamp                  12
sin_time                   13
cos_time                   14
discrete_y                 15
glucose_level_indicate     16
finger_stick_indicate      17
blood_ketone_indicate      18
meal_indicate              19
insulin_dose_sc_indicate   20
hypo_agents_indicate       21
insulin_bolus_indicate     22
basal_indicate             23
insulin_iv_indicate        24
bolus_indicate             25
                        attri
idx                          
0                finger_stick
1                blood_ketone
2                        meal
3         

In [26]:
for pid in pid_list:
    pid2train_norm[pid] = pid2train_norm[pid].fillna(0.0)
    pid2valid_norm[pid] = pid2valid_norm[pid].fillna(0.0)
    pid2test_norm[pid] = pid2test_norm[pid].fillna(0.0)

In [27]:
# generate training data


def gen_final(pid2data, attri2idx,  n_prev, pred_window, name, valid_points = 24): 
    pid2data_npy = {}
    for pid in pid_list:
        data = pid2data[pid]
        
        y_list = []
        y_dis_list = []
        target_cos_time_list = []
        target_sin_time_list = []
        target_total_seconds_list = []
        attri_X_list = []
        attri_X_tar_list = []
        y_tar_list = []
        glucose_level_X_list = []

        seq_st_ed_list = []
        
        seq_st, seq_ed = None, None
        
        for idx in range(len(data) - n_prev - pred_window + 1):

            tm_idx = data.iloc[idx: idx + n_prev + pred_window].index
            tm_delta = (tm_idx[1:] - tm_idx[:-1]).total_seconds()/60/60
            tm_delta_max = tm_delta.max()
            if tm_delta_max > 2:
                if seq_st is not None and seq_ed - seq_st >= 2:
                    seq_st_ed_list.append([seq_st, seq_ed])
                seq_st, seq_ed = None, None
                continue

            glucose_level_y = data.iloc[idx + n_prev + pred_window - 1]['glucose_level']
            glucose_level_y_indicate = data.iloc[idx + n_prev + pred_window - 1]['glucose_level_indicate']
            if abs(glucose_level_y_indicate - 1) < 1e-6:
                continue

            # Resonable
            num_valids = int(valid_points - sum(data.iloc[idx + n_prev - valid_points : idx + n_prev]['glucose_level_indicate']))
            if num_valids == 0:
                if seq_st is not None and seq_ed - seq_st >= 2:
                    seq_st_ed_list.append([seq_st, seq_ed])
                seq_st, seq_ed = None, None
                continue
            # end

            
            y_list.append(glucose_level_y)
            y_dis_list.append(data.iloc[idx + n_prev + pred_window - 1]['discrete_y'])
            target_cos_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['cos_time'])
            target_sin_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['sin_time'])
            target_total_seconds_list.append(data.iloc[idx + n_prev + pred_window - 1]['timestamp'])
            glucose_level_X = np.array(data.iloc[idx: idx + n_prev]['glucose_level'].to_numpy(), dtype=np.float32)


            attri_X = np.array(data.iloc[idx: idx + n_prev][list(attri2idx.index)].to_numpy(), dtype=np.float32)          

            glucose_level_X_list.append(np.expand_dims(glucose_level_X, axis=0))

            attri_X_list.append(np.expand_dims(attri_X, axis=0))
            # new
            attri_X_tar = np.array(data.iloc[idx + n_prev: idx + n_prev + pred_window][list(attri2idx.index)].to_numpy(), dtype=np.float32)          
            attri_X_tar_list.append(np.expand_dims(attri_X_tar, axis=0))
            y_tar = data.iloc[idx + n_prev: idx + n_prev + pred_window]['glucose_level']
            y_tar_list.append(np.expand_dims(y_tar, axis=0)) 
            if seq_st is None:
                seq_st = len(y_list) - 1
            else:
                seq_ed = len(y_list) - 1

        if seq_st is not None and seq_ed - seq_st >= 2:
            seq_st_ed_list.append([seq_st, seq_ed])

        pid2data_npy[pid] = {}
        pid2data_npy[pid]['glucose_level_X'] = np.concatenate(glucose_level_X_list, axis=0)

        pid2data_npy[pid]['attri_X'] = np.concatenate(attri_X_list, axis=0)
        pid2data_npy[pid]['y'] = np.array(y_list, dtype=np.float32)
        pid2data_npy[pid]['y_dis'] = np.array(y_dis_list, dtype=np.int32)
        pid2data_npy[pid]['target_cos_time'] = np.array(target_cos_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_sin_time'] = np.array(target_sin_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_timestamp'] = np.array(target_total_seconds_list, dtype=np.float32)
        pid2data_npy[pid]['mean'] = pid_attri2mean_std[(pid, 'glucose_level')][0]
        pid2data_npy[pid]['std'] = pid_attri2mean_std[(pid, 'glucose_level')][1]
        pid2data_npy[pid]['attri_X_tar'] = np.concatenate(attri_X_tar_list, axis=0) 
        pid2data_npy[pid]['y_tar'] = np.concatenate(y_tar_list, axis=0)     
        pid2data_npy[pid]['seq_st_ed_list'] = np.array(seq_st_ed_list, dtype=np.int32)
        print(pid, pid2data_npy[pid]['glucose_level_X'].shape, pid2data_npy[pid]['attri_X'].shape)
        print('    ',len(y_list), seq_st_ed_list)
    save_path = os.path.join(data_path, process_file)
    np.save(os.path.join(save_path, f'{name}_{n_prev}_{pred_window}.npy'), pid2data_npy)




In [28]:
gen_final(pid2train_norm, attri2idx,  n_prev=16, pred_window=2, name='train_pid2data_npy', valid_points = 8)

2000 (787, 16) (787, 16, 26)
     787 [[0, 786]]
2001 (1487, 16) (1487, 16, 26)
     1487 [[0, 1177], [1178, 1486]]
2002 (326, 16) (326, 16, 26)
     326 [[0, 325]]
2003 (335, 16) (335, 16, 26)
     335 [[0, 334]]
2004 (274, 16) (274, 16, 26)
     274 [[0, 273]]
2005 (398, 16) (398, 16, 26)
     398 [[0, 397]]
2006 (272, 16) (272, 16, 26)
     272 [[0, 271]]
2007 (482, 16) (482, 16, 26)
     482 [[0, 481]]
2008 (303, 16) (303, 16, 26)
     303 [[0, 302]]
2009 (280, 16) (280, 16, 26)
     280 [[0, 279]]
2010 (307, 16) (307, 16, 26)
     307 [[0, 306]]
2011 (261, 16) (261, 16, 26)
     261 [[0, 260]]
2012 (156, 16) (156, 16, 26)
     156 [[0, 155]]
2013 (194, 16) (194, 16, 26)
     194 [[0, 193]]
2014 (1570, 16) (1570, 16, 26)
     1570 [[0, 1315], [1316, 1569]]
2015 (1561, 16) (1561, 16, 26)
     1561 [[0, 1321], [1322, 1560]]
2016 (787, 16) (787, 16, 26)
     787 [[0, 786]]
2017 (1571, 16) (1571, 16, 26)
     1571 [[0, 1319], [1320, 1570]]
2018 (722, 16) (722, 16, 26)
     722 [[0, 721

In [29]:
gen_final(pid2train_norm, attri2idx,  n_prev=16, pred_window=4, name='train_pid2data_npy', valid_points = 8)

2000 (785, 16) (785, 16, 26)
     785 [[0, 784]]
2001 (1483, 16) (1483, 16, 26)
     1483 [[0, 1175], [1176, 1482]]
2002 (324, 16) (324, 16, 26)
     324 [[0, 323]]
2003 (333, 16) (333, 16, 26)
     333 [[0, 332]]
2004 (272, 16) (272, 16, 26)
     272 [[0, 271]]
2005 (396, 16) (396, 16, 26)
     396 [[0, 395]]
2006 (270, 16) (270, 16, 26)
     270 [[0, 269]]
2007 (480, 16) (480, 16, 26)
     480 [[0, 479]]
2008 (301, 16) (301, 16, 26)
     301 [[0, 300]]
2009 (278, 16) (278, 16, 26)
     278 [[0, 277]]
2010 (305, 16) (305, 16, 26)
     305 [[0, 304]]
2011 (259, 16) (259, 16, 26)
     259 [[0, 258]]
2012 (154, 16) (154, 16, 26)
     154 [[0, 153]]
2013 (192, 16) (192, 16, 26)
     192 [[0, 191]]
2014 (1566, 16) (1566, 16, 26)
     1566 [[0, 1313], [1314, 1565]]
2015 (1557, 16) (1557, 16, 26)
     1557 [[0, 1319], [1320, 1556]]
2016 (785, 16) (785, 16, 26)
     785 [[0, 784]]
2017 (1567, 16) (1567, 16, 26)
     1567 [[0, 1317], [1318, 1566]]
2018 (720, 16) (720, 16, 26)
     720 [[0, 719

In [30]:
gen_final(pid2valid_norm, attri2idx,  n_prev=16, pred_window=2, name='valid_pid2data_npy', valid_points = 8)

2000 (251, 16) (251, 16, 26)
     251 [[0, 250]]
2001 (490, 16) (490, 16, 26)
     490 [[0, 489]]
2002 (97, 16) (97, 16, 26)
     97 [[0, 96]]
2003 (100, 16) (100, 16, 26)
     100 [[0, 99]]
2004 (80, 16) (80, 16, 26)
     80 [[0, 79]]
2005 (121, 16) (121, 16, 26)
     121 [[0, 120]]
2006 (79, 16) (79, 16, 26)
     79 [[0, 78]]
2007 (149, 16) (149, 16, 26)
     149 [[0, 148]]
2008 (90, 16) (90, 16, 26)
     90 [[0, 89]]
2009 (82, 16) (82, 16, 26)
     82 [[0, 81]]
2010 (91, 16) (91, 16, 26)
     91 [[0, 90]]
2011 (75, 16) (75, 16, 26)
     75 [[0, 74]]
2012 (41, 16) (41, 16, 26)
     41 [[0, 40]]
2013 (53, 16) (53, 16, 26)
     53 [[0, 52]]
2014 (517, 16) (517, 16, 26)
     517 [[0, 516]]
2015 (514, 16) (514, 16, 26)
     514 [[0, 513]]
2016 (251, 16) (251, 16, 26)
     251 [[0, 250]]
2017 (518, 16) (518, 16, 26)
     518 [[0, 517]]
2018 (229, 16) (229, 16, 26)
     229 [[0, 228]]
2019 (133, 16) (133, 16, 26)
     133 [[0, 132]]
2020 (91, 16) (91, 16, 26)
     91 [[0, 90]]
2021 (210, 1

In [31]:
gen_final(pid2valid_norm, attri2idx,  n_prev=16, pred_window=4, name='valid_pid2data_npy', valid_points = 8)

2000 (249, 16) (249, 16, 26)
     249 [[0, 248]]
2001 (488, 16) (488, 16, 26)
     488 [[0, 487]]
2002 (95, 16) (95, 16, 26)
     95 [[0, 94]]
2003 (98, 16) (98, 16, 26)
     98 [[0, 97]]
2004 (78, 16) (78, 16, 26)
     78 [[0, 77]]
2005 (119, 16) (119, 16, 26)
     119 [[0, 118]]
2006 (77, 16) (77, 16, 26)
     77 [[0, 76]]
2007 (147, 16) (147, 16, 26)
     147 [[0, 146]]
2008 (88, 16) (88, 16, 26)
     88 [[0, 87]]
2009 (80, 16) (80, 16, 26)
     80 [[0, 79]]
2010 (89, 16) (89, 16, 26)
     89 [[0, 88]]
2011 (73, 16) (73, 16, 26)
     73 [[0, 72]]
2012 (39, 16) (39, 16, 26)
     39 [[0, 38]]
2013 (51, 16) (51, 16, 26)
     51 [[0, 50]]
2014 (515, 16) (515, 16, 26)
     515 [[0, 514]]
2015 (512, 16) (512, 16, 26)
     512 [[0, 511]]
2016 (249, 16) (249, 16, 26)
     249 [[0, 248]]
2017 (516, 16) (516, 16, 26)
     516 [[0, 515]]
2018 (227, 16) (227, 16, 26)
     227 [[0, 226]]
2019 (131, 16) (131, 16, 26)
     131 [[0, 130]]
2020 (89, 16) (89, 16, 26)
     89 [[0, 88]]
2021 (208, 16) 

In [32]:
gen_final(pid2test_norm, attri2idx,  n_prev=16, pred_window=2, name='test_pid2data_npy', valid_points = 8)

2000 (250, 16) (250, 16, 26)
     250 [[0, 249]]
2001 (489, 16) (489, 16, 26)
     489 [[0, 488]]
2002 (97, 16) (97, 16, 26)
     97 [[0, 96]]
2003 (99, 16) (99, 16, 26)
     99 [[0, 98]]
2004 (79, 16) (79, 16, 26)
     79 [[0, 78]]
2005 (121, 16) (121, 16, 26)
     121 [[0, 120]]
2006 (78, 16) (78, 16, 26)
     78 [[0, 77]]
2007 (149, 16) (149, 16, 26)
     149 [[0, 148]]
2008 (89, 16) (89, 16, 26)
     89 [[0, 88]]
2009 (81, 16) (81, 16, 26)
     81 [[0, 80]]
2010 (90, 16) (90, 16, 26)
     90 [[0, 89]]
2011 (75, 16) (75, 16, 26)
     75 [[0, 74]]
2012 (40, 16) (40, 16, 26)
     40 [[0, 39]]
2013 (52, 16) (52, 16, 26)
     52 [[0, 51]]
2014 (517, 16) (517, 16, 26)
     517 [[0, 516]]
2015 (514, 16) (514, 16, 26)
     514 [[0, 513]]
2016 (250, 16) (250, 16, 26)
     250 [[0, 249]]
2017 (517, 16) (517, 16, 26)
     517 [[0, 516]]
2018 (229, 16) (229, 16, 26)
     229 [[0, 228]]
2019 (133, 16) (133, 16, 26)
     133 [[0, 132]]
2020 (90, 16) (90, 16, 26)
     90 [[0, 89]]
2021 (209, 16) 

In [33]:
gen_final(pid2test_norm, attri2idx,  n_prev=16, pred_window=4, name='test_pid2data_npy', valid_points = 8)

2000 (248, 16) (248, 16, 26)
     248 [[0, 247]]
2001 (487, 16) (487, 16, 26)
     487 [[0, 486]]
2002 (95, 16) (95, 16, 26)
     95 [[0, 94]]
2003 (97, 16) (97, 16, 26)
     97 [[0, 96]]
2004 (77, 16) (77, 16, 26)
     77 [[0, 76]]
2005 (119, 16) (119, 16, 26)
     119 [[0, 118]]
2006 (76, 16) (76, 16, 26)
     76 [[0, 75]]
2007 (147, 16) (147, 16, 26)
     147 [[0, 146]]
2008 (87, 16) (87, 16, 26)
     87 [[0, 86]]
2009 (79, 16) (79, 16, 26)
     79 [[0, 78]]
2010 (88, 16) (88, 16, 26)
     88 [[0, 87]]
2011 (73, 16) (73, 16, 26)
     73 [[0, 72]]
2012 (38, 16) (38, 16, 26)
     38 [[0, 37]]
2013 (50, 16) (50, 16, 26)
     50 [[0, 49]]
2014 (515, 16) (515, 16, 26)
     515 [[0, 514]]
2015 (512, 16) (512, 16, 26)
     512 [[0, 511]]
2016 (248, 16) (248, 16, 26)
     248 [[0, 247]]
2017 (515, 16) (515, 16, 26)
     515 [[0, 514]]
2018 (227, 16) (227, 16, 26)
     227 [[0, 226]]
2019 (131, 16) (131, 16, 26)
     131 [[0, 130]]
2020 (88, 16) (88, 16, 26)
     88 [[0, 87]]
2021 (207, 16) 